# core

>Here's where the main `formalyzer` workflow is defined

```markdown
formalyzer: 

Reads PDF reccomendation letter, fills in admissions form(s)

usage: 
  formalyzer <recc_letter.pdf> <url_list.txt>

Instead of url_list.txt, a single URL can be given (esp. for testing purposes) 

Description: 
Formalyzer will scrape the text from the PDF recc letter, 
and for each URL in url_list, it will: 
- launch a browser tab for that url 
- fill in the form using what the LLM has gleaned from the recc letter
- attach the PDF via the form's upload/attachment button
...and do no more. 
The user will need to review the page and press the Submit button manually.


Requirements: 
- Playwright 
- ANTHROPIC_API_KEY env var. (Could support other LLMs layer)
- pypdf  

Author: Scott H. Hawley, @drscotthawley
```



In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os 

def read_recc_info(info_file:str) -> list:
    "read a text file of info on the reviewer" 
    with open(os.path.expanduser(info_file)) as f:
        return f.read()

In [ ]:
recc_info = read_recc_info("~/recc_info.txt") 
recc_info

'Reccomender Name: Scott H. Hawley \nTitle: Professor of Physics \n\nAddress: \nBelmont University \n1900 Belmont Blvd \nNashville, TN 37211\n\nPhone: 615-460-6206\nEmail: scott.hawley@belmont.edu\n'

In [ ]:
#| export
def read_urls_file(urls_file:str) -> list:
    "read a text file where each line is a url of a submission site" 
    with open(os.path.expanduser(urls_file)) as f:
        return f.read().splitlines()

In [ ]:
urls = read_urls_file("~/recc_urls.txt") 
print(f"{len(urls)} urls in list")

11 urls in list


In [ ]:
#| export
from pypdf import PdfReader
import logging
logging.getLogger("pypdf").setLevel(logging.ERROR)

def read_pdf_text(pdf_file):
    reader = PdfReader(os.path.expanduser(pdf_file))
    return "\n".join(page.extract_text() for page in reader.pages)

In [ ]:
letter_text = read_pdf_text("~/recc_letter.pdf")
#letter_text

Okay, we've got the basic functionality where we can read in the letter as $`letter_text` and the url list as $`urls`. 
Now, what we want to do is pick the first URL and let's try to open a browser tab using using Playwright at that URL. Ideally we'd be doing this from my laptop browser not your server, so we might need to do some fancy stuff like ssh tunneling. What do you think? Refer to the lesson Transcript above where Jeremy was using Playwright to control his browser. 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Great question! Yes, based on Jeremy's approach in the transcript, controlling your *local* browser is definitely the way to go here — especially since these are authenticated admissions portals that likely need you to be logged in or will have anti-bot measures.

Here's the rough plan from the lesson:

1. **On your laptop**: Start Chrome with remote debugging enabled on port 9222
2. **Create a tunnel**: Use bore.pub, ngrok, or similar so solve it can reach your laptop
3. **SSH with port forwarding**: Connect from solve it to your laptop, forwarding port 9222
4. **Use Playwright's `connect_over_cdp`**: To control your local browser from here

A few questions before we dive in:

1. **Do you already have a tunneling service set up** (like bore.pub, ngrok, or similar)?
2. **What OS is your laptop** — Mac, Windows, or Linux? (This affects how we launch Chrome with debugging)
3. **Have you set up SSH keys** so solve it can connect to your laptop without a password?

If you haven't done the tunneling/SSH setup yet, that's the first step. If you *have* done it (or want to skip that for now and just test Playwright locally on the server with a headless browser first), let me know and we can proceed accordingly!

I have a Mac.  I've used `bore` and `ngrok`.  both are installed.   I have my ssh keys setup already between solveit and my laptop. Walk me through the steps. 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Excellent — you're all set up! Let's do this step by step.

**Step 1: Start Chrome with remote debugging on your Mac**

In your Mac terminal, run:

```bash
/Applications/Google\ Chrome.app/Contents/MacOS/Google\ Chrome --remote-debugging-port=9222
```

This will open Chrome with the debugging port enabled. (If you want a separate profile to avoid conflicts with your normal Chrome, you can add `--user-data-dir=/tmp/chrome-debug` to the command.)

Let me know once Chrome is running with that command — then we'll move to Step 2 (starting the tunnel).

Okay, Chrome is running. I don't really care about the user data, but I think in the newer versions, it's actually required.  So I ran `/Applications/Google\ Chrome.app/Contents/MacOS/Google\ Chrome --remote-debugging-port=9222 --user-data-dir=/tmp/chrome-debug` and clicked through all the menus so it's ready to go. 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Perfect! You're right about the `--user-data-dir` being required in newer Chrome versions.

**Step 2: Start the tunnel to expose your SSH port**

On your Mac, in a *separate* terminal window, run:

```bash
bore local 22 --to bore.pub
```

This will expose your Mac's SSH port (22) to the internet via bore.pub. It'll print something like:

```
Listening at bore.pub:XXXXX
```

Note down that port number (the XXXXX part) — we'll need it for the next step.

Let me know the port once bore is running!

bore.pub:14917

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Great! 

**Step 3: SSH from solveit to your Mac with port forwarding**

Now we need to connect from solveit to your Mac, forwarding port 9222 (Chrome's debugging port). Run this in a code cell here:

```python
import subprocess
subprocess.Popen([
    "ssh", "-N", "-L", "9222:localhost:9222",
    "-p", "14917",
    "YOUR_MAC_USERNAME@bore.pub"
])
```

Replace `YOUR_MAC_USERNAME` with your Mac username (the one you'd use to SSH into your laptop).

The `-N` means "don't run a command, just forward the port" and `-L 9222:localhost:9222` sets up the tunnel so that localhost:9222 on solveit connects to localhost:9222 on your Mac.

What's your Mac username? (Or just run it with the correct username and let me know if it connects!)

In [ ]:
import subprocess
subprocess.Popen([
    "ssh", "-N", "-L", "9222:localhost:9222",
    "-p", "14917",
    "shawley@bore.pub"
])

<Popen: returncode: None args: ['ssh', '-N', '-L', '9222:localhost:9222', '-...>

How's that? Are we good to go on? 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Let's verify the tunnel is working before we proceed. Run this quick test:

```python
import socket
s = socket.socket()
try:
    s.connect(('localhost', 9222))
    print("✓ Port 9222 is reachable!")
    s.close()
except Exception as e:
    print(f"✗ Connection failed: {e}")
```

If that shows the port is reachable, we're ready for Step 4: connecting Playwright to your browser!

In [ ]:
import socket
s = socket.socket()
try:
    s.connect(('localhost', 9222))
    print("✓ Port 9222 is reachable!")
    s.close()
except Exception as e:
    print(f"✗ Connection failed: {e}")

✓ Port 9222 is reachable!


What next? 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Excellent! The tunnel is working. 

**Step 4: Connect Playwright to your browser**

```python
from playwright.async_api import async_playwright

pw = await async_playwright().start()
browser = await pw.chromium.connect_over_cdp("http://localhost:9222")
page = await browser.new_page()
```

Run that, and if it works, you should see a new blank tab appear in Chrome on your Mac! Then we can navigate to the first URL from your list.w

In [ ]:
from playwright.async_api import async_playwright

pw = await async_playwright().start()
browser = await pw.chromium.connect_over_cdp("http://localhost:9222")
page = await browser.new_page()

Yep! Got a new browser page.  Now how about if we make it navigate to $`urls[0]`? 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Perfect! Try this:

```python
await page.goto(urls[0])
```

This should navigate to the first URL in your list. Let me know what you see — does the page load?

In [ ]:
await page.goto(urls[0])

<Response url='https://apply.grad.ucla.edu/apply/refer?key=0229711527540226' request=<Request url='https://apply.grad.ucla.edu/apply/refer?key=0229711527540226' method='GET'>>

great okay Looks like this is working. Now what I want you to do is take the recommender info in $`recc_info`, And on the form, fill in all the recommender info, e.g. name, address, etc. intelligently as a LLM. this implies that somehow you've scraped the page and you know what the form fields are.  In the cells above, I've included two examples of HTML source from sample pages, although Although really you should scrape to find out what's in there. the plan is to use `playwright` to do the form filling. 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Great! Let's approach this step by step. First, let's get the page content to see what form fields are on the current page:

```python
html = await page.content()
print(f"Page length: {len(html)} chars")
```

Once we see what's there, we can identify the input fields and fill them in using the `recc_info`. We'll use Playwright's `fill()` and `select_option()` methods to populate the form based on matching the field labels/names to the recommender data.

Run that first and let me know what you get — then we can write the form-filling logic!

In [ ]:
html = await page.content()
print(f"Page length: {len(html)} chars")

Page length: 190157 chars


.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Good, we have the HTML. Now let's parse it to find the form fields. Let's extract the input fields and their labels:

```python
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

# Find all text inputs, selects, and textareas
fields = []
for inp in soup.find_all(['input', 'select', 'textarea']):
    if inp.get('type') in ['hidden', 'submit', 'button', 'radio', 'checkbox', 'file']:
        continue
    field_id = inp.get('id', '')
    field_name = inp.get('name', '')
    
    # Try to find associated label
    label = soup.find('label', {'for': field_id})
    label_text = label.get_text(strip=True) if label else ''
    
    current_value = inp.get('value', '')
    
    fields.append({
        'id': field_id,
        'name': field_name,
        'label': label_text,
        'tag': inp.name,
        'value': current_value
    })

for f in fields[:15]:  # Show first 15
    print(f"{f['label'][:40]:<40} | id={f['id'][:30]} | val={f['value'][:20] if f['value'] else ''}")
```

This will show us what fields we're working with. Run that and let's see the structure!

Oh, so Playwright doesn't do that stuff automatically for us. we have to run beautiful soup to parse it?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Good question! Playwright *does* have built-in ways to find and interact with elements — you don't *need* BeautifulSoup. Playwright offers:

- `page.locator('input[name="fieldname"]')` — CSS selectors
- `page.get_by_label("First Name")` — find by label text
- `page.get_by_placeholder("Enter name")` — find by placeholder
- `page.fill('#field_id', 'value')` — fill by ID

So we have two approaches:

1. **BeautifulSoup first**: Parse the HTML to understand what fields exist, then use Playwright to fill them
2. **Pure Playwright**: Use `get_by_label()` to find fields directly by their label text

For a one-off form fill, approach 2 is simpler. For building a reusable tool that handles *many different* forms, approach 1 (understanding the structure first) is more robust.

Which approach appeals to you? Or shall we try the Playwright-native approach first and see how far we get?

definelly #1. The whole reason for this app is that there are many different forms. I have 14 different URLs from different schools, and they're all different. different. And so that's why I want the LLM's help. It might even be helpful to take screenshots. However, a number of these forms take up more than one page. So that's why I was initially thinking we scrape. 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

That makes total sense — scraping is the right approach for a multi-form tool. The flow would be:

1. **Scrape the HTML** → extract all form fields (id, name, label, type)
2. **Feed to LLM** → "Here are the fields, here's the recommender info, tell me how to fill each field"
3. **Use Playwright** → fill the fields based on LLM's mapping
4. **Handle pagination** → detect "Next" buttons, repeat for each page

For step 1, shall I write the BeautifulSoup parsing code to extract a clean list of form fields? That'll give us a structured view we can then send to the LLM for intelligent mapping.

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

# Find all text inputs, selects, and textareas
fields = []
for inp in soup.find_all(['input', 'select', 'textarea']):
    if inp.get('type') in ['hidden', 'submit', 'button', 'radio', 'checkbox', 'file']:
        continue
    field_id = inp.get('id', '')
    field_name = inp.get('name', '')
    
    # Try to find associated label
    label = soup.find('label', {'for': field_id})
    label_text = label.get_text(strip=True) if label else ''
    
    current_value = inp.get('value', '')
    
    fields.append({
        'id': field_id,
        'name': field_name,
        'label': label_text,
        'tag': inp.name,
        'value': current_value
    })

for f in fields[:15]:  # Show first 15
    print(f"{f['label'][:40]:<40} | id={f['id'][:30]} | val={f['value'][:20] if f['value'] else ''}")

                                         | id= | val=
                                         | id=form_b2d46a9c-36e6-4095-bf56-9 | val=
First Name*                              | id=form_05362460-be07-f890-f2c5-c | val=Scott
Last Name*                               | id=form_8644da3f-60e7-4a78-f6e2-7 | val=Hawley
Position or Title*                       | id=form_774484fe-dd73-5ebe-6ac3-1 | val=Professor
Name of Institution or Business*         | id=form_b913c5b9-8a40-dc55-d14d-d | val=Belmont University
Institution or Business Address 1        | id=form_d4a7d23d-9fd3-5542-466e-d | val=
Institution or Business Address 2        | id=form_210773b2-503d-7748-63b5-f | val=
City                                     | id=form_cb198687-7870-debf-65f3-8 | val=
State/Province/Region                    | id=form_1f9293f9-1537-56e3-7bb2-0 | val=
Zip Code                                 | id=form_a06b225f-d0e7-701d-34c7-e | val=
Country*                                 | id=form_c3e60866-2c14-426

Okay, so it looks like some of these fields are pre-filled out on the site and we don't need to change their values. That's kind of good. Now, the intellectual ability thing, you're going to need to fill in yourself based on your read of the $`letter_text`....And there are a bunch more fields after that that similarly will need to be filled in from the letter. And somewhere down there is a button to attach a file for which I want to attach the PDF.  

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Exactly right! Let's get the full picture. Let's see all the fields, including the selects (dropdowns) and the file input:

```python
# Get ALL fields including selects with options, and file inputs
for inp in soup.find_all(['input', 'select', 'textarea']):
    field_id = inp.get('id', '')
    field_type = inp.get('type', inp.name)
    label = soup.find('label', {'for': field_id})
    label_text = label.get_text(strip=True) if label else ''
    current_value = inp.get('value', '')
    
    # For selects, show the options
    if inp.name == 'select':
        options = [opt.get_text(strip=True) for opt in inp.find_all('option') if opt.get_text(strip=True)]
        print(f"[SELECT] {label_text[:50]}")
        print(f"   Options: {options[:5]}...")  # First 5 options
    elif field_type == 'file':
        print(f"[FILE] {label_text} | id={field_id}")
    elif current_value:
        print(f"[PREFILLED] {label_text}: {current_value}")
    else:
        print(f"[EMPTY] {label_text} | id={field_id}")
```

This will show us:
- What's already filled (we can skip)
- The dropdown options (so the LLM knows what choices to pick)
- The file upload field

Run that and we'll see the full scope of what needs doing!

In [ ]:
# Get ALL fields including selects with options, and file inputs
for inp in soup.find_all(['input', 'select', 'textarea']):
    field_id = inp.get('id', '')
    field_type = inp.get('type', inp.name)
    label = soup.find('label', {'for': field_id})
    label_text = label.get_text(strip=True) if label else ''
    current_value = inp.get('value', '')
    
    # For selects, show the options
    if inp.name == 'select':
        options = [opt.get_text(strip=True) for opt in inp.find_all('option') if opt.get_text(strip=True)]
        print(f"[SELECT] {label_text[:50]}")
        print(f"   Options: {options[:5]}...")  # First 5 options
    elif field_type == 'file':
        print(f"[FILE] {label_text} | id={field_id}")
    elif current_value:
        print(f"[PREFILLED] {label_text}: {current_value}")
    else:
        print(f"[EMPTY] {label_text} | id={field_id}")

[EMPTY]  | id=
[PREFILLED] : 02297115
[PREFILLED] : 27540226
[SELECT] 
   Options: ['Nursing DNP ONLY', 'Nursing MECN ONLY', 'Nursing PHD ONLY', 'Standard']...
[PREFILLED] First Name*: Scott
[PREFILLED] Last Name*: Hawley
[PREFILLED] Position or Title*: Professor
[PREFILLED] Name of Institution or Business*: Belmont University
[EMPTY] Institution or Business Address 1 | id=form_d4a7d23d-9fd3-5542-466e-d2e30fe30e6c
[EMPTY] Institution or Business Address 2 | id=form_210773b2-503d-7748-63b5-f0f37c83b6ed
[EMPTY] City | id=form_cb198687-7870-debf-65f3-8daf444c1a7f
[EMPTY] State/Province/Region | id=form_1f9293f9-1537-56e3-7bb2-0f4d3a1848a5
[EMPTY] Zip Code | id=form_a06b225f-d0e7-701d-34c7-e92c5b4d3f3e
[SELECT] Country*
   Options: ['Afghanistan', 'Aland Islands', 'Albania', 'Algeria', 'Andorra']...
[PREFILLED] Phone Number*: +1 615-460-6206
[EMPTY] Email* | id=form_e75dd53c-9b7a-718f-e819-4073d1733059
[SELECT] Intellectual Ability*
   Options: ['Outstanding', 'Excellent', 'Very Good', 'Av

Okay, so for most of those questions about the person, you're going to need to fill those in based on the $`letter_text`. Next. Anything else I want you to guess. For example, the $`recc_info` doesn't explicitly say that I live in the USA, but you can see from "Nashville, TN", that knowing that that's in the USA, that ought to help. Make sense? 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Yes, makes total sense! So the approach is:

1. **From `recc_info`**: Fill address fields (with inference like TN → Tennessee, USA)
2. **From `letter_text`**: Infer the ratings — the letter says "top 2% of students" and "excellent" work, so that maps to "Outstanding" for most categories
3. **Smart defaults**: For things not mentioned (like "Laboratory Skills"), maybe "Insufficient Knowledge, N/A"

Let me build a clean structure of what needs filling, then we can make an LLM call to generate the mappings. First, let's extract just the fields that need values:

```python
fields_to_fill = []
for inp in soup.find_all(['input', 'select', 'textarea']):
    field_id = inp.get('id', '')
    if not field_id: continue
    field_type = inp.get('type', inp.name)
    if field_type in ['hidden', 'submit', 'button']: continue
    
    label = soup.find('label', {'for': field_id})
    label_text = label.get_text(strip=True) if label else ''
    current_value = inp.get('value', '')
    
    # Skip if already has a meaningful value (not just radio button values)
    if current_value and field_type not in ['radio', 'checkbox'] and inp.name != 'select':
        continue
        
    options = None
    if inp.name == 'select':
        options = [opt.get_text(strip=True) for opt in inp.find_all('option') if opt.get_text(strip=True)]
    
    fields_to_fill.append({
        'id': field_id, 'label': label_text, 'type': field_type, 'options': options
    })

print(f"Found {len(fields_to_fill)} fields to fill")
```

Run that and we'll see the clean list of what needs doing!

In [ ]:
fields_to_fill = []
for inp in soup.find_all(['input', 'select', 'textarea']):
    field_id = inp.get('id', '')
    if not field_id: continue
    field_type = inp.get('type', inp.name)
    if field_type in ['hidden', 'submit', 'button']: continue
    
    label = soup.find('label', {'for': field_id})
    label_text = label.get_text(strip=True) if label else ''
    current_value = inp.get('value', '')
    
    # Skip if already has a meaningful value (not just radio button values)
    if current_value and field_type not in ['radio', 'checkbox'] and inp.name != 'select':
        continue
        
    options = None
    if inp.name == 'select':
        options = [opt.get_text(strip=True) for opt in inp.find_all('option') if opt.get_text(strip=True)]
    
    fields_to_fill.append({
        'id': field_id, 'label': label_text, 'type': field_type, 'options': options
    })

print(f"Found {len(fields_to_fill)} fields to fill")

Found 123 fields to fill


ok?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Yes! We have 123 fields to fill. Now let's create a prompt for the LLM to map the recommender info and letter text to these fields. Here's what I suggest:

```python
import json

# Create a structured prompt for the LLM
prompt = f"""You are filling out a graduate school recommendation form.

RECOMMENDER INFO:
{recc_info}

RECOMMENDATION LETTER:
{letter_text[:2000]}... [truncated]

FORM FIELDS TO FILL (first 20):
{json.dumps(fields_to_fill[:20], indent=2)}

For each field, provide:
1. The field ID
2. The value to fill in (or select from options)
3. Brief reasoning

Return as JSON array like:
[{{"id": "form_xxx", "value": "Outstanding", "reason": "Letter says top 2%"}}]
"""

print(f"Prompt length: {len(prompt)} chars")
print(prompt[:500])
```

This will show us what we're sending to the LLM. Then we can make the actual API call. Want to see the prompt first?

In [ ]:
import json

# Create a structured prompt for the LLM
prompt = f"""You are filling out a graduate school recommendation form.

RECOMMENDER INFO:
{recc_info}

RECOMMENDATION LETTER:
{letter_text[:2000]}... [truncated]

FORM FIELDS TO FILL (first 20):
{json.dumps(fields_to_fill[:20], indent=2)}

For each field, provide:
1. The field ID
2. The value to fill in (or select from options)
3. Brief reasoning

Return as JSON array like:
[{{"id": "form_xxx", "value": "Outstanding", "reason": "Letter says top 2%"}}]
"""

print(f"Prompt length: {len(prompt)} chars")
print(prompt[:500])

Prompt length: 12995 chars
You are filling out a graduate school recommendation form.

RECOMMENDER INFO:
Reccomender Name: Scott H. Hawley 
Title: Professor of Physics 

Address: 
Belmont University 
1900 Belmont Blvd 
Nashville, TN 37211

Phone: 615-460-6206
Email: scott.hawley@belmont.edu


RECOMMENDATION LETTER:
  Dear Graduate Admissions Committee,  It is with absolute confidence that I strongly recommend Brody Blackwood for admission to your program. He is extremely well-suited for graduate work and ranks within the 


Next? 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

**Note:** Some earlier messages were truncated due to context limits, but we have what we need.

Now let's call the LLM to get the field mappings. We'll use claudette:

```python
from claudette import Chat

chat = Chat(model="claude-sonnet-4-20250514")
response = chat(prompt)
print(response)
```

This will send the prompt with the recommender info, letter, and field structure to Claude and get back the JSON mapping. Run that and let's see what it suggests!

In [ ]:
from claudette import Chat

chat = Chat(model="claude-sonnet-4-20250514")
response = chat(prompt)
print(response)

Message(id='msg_01KAg9RcNiQJa2PrputhLozs', content=[TextBlock(citations=None, text='```json\n[\n  {\n    "id": "form_b2d46a9c-36e6-4095-bf56-9d15b524bea5",\n    "value": "Standard",\n    "reason": "This is a physics student recommendation, not nursing-specific"\n  },\n  {\n    "id": "form_d4a7d23d-9fd3-5542-466e-d2e30fe30e6c",\n    "value": "Belmont University",\n    "reason": "Primary institution address from recommender info"\n  },\n  {\n    "id": "form_210773b2-503d-7748-63b5-f0f37c83b6ed",\n    "value": "1900 Belmont Blvd",\n    "reason": "Street address from recommender info"\n  },\n  {\n    "id": "form_cb198687-7870-debf-65f3-8daf444c1a7f",\n    "value": "Nashville",\n    "reason": "City from recommender address"\n  },\n  {\n    "id": "form_1f9293f9-1537-56e3-7bb2-0f4d3a1848a5",\n    "value": "TN",\n    "reason": "State from recommender address"\n  },\n  {\n    "id": "form_a06b225f-d0e7-701d-34c7-e92c5b4d3f3e",\n    "value": "37211",\n    "reason": "Zip code from recommender addr

That's looking pretty good, although I'm not sure if it included all of the fields. 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

You're right — we only sent the first 20 fields in the prompt (`fields_to_fill[:20]`). We have 123 total! Let's send all of them:

```python
# Send ALL fields this time
prompt_full = f"""You are filling out a graduate school recommendation form.

RECOMMENDER INFO:
{recc_info}

RECOMMENDATION LETTER:
{letter_text}

FORM FIELDS TO FILL:
{json.dumps(fields_to_fill, indent=2)}

For each field, provide the field ID and value to fill. For dropdowns, pick from the options listed.
Return as JSON array: [{{"id": "form_xxx", "value": "..."}}]
Skip fields that are already filled or radio buttons.
"""

print(f"Full prompt length: {len(prompt_full)} chars")
response_full = chat(prompt_full)
print(response_full)
```

This sends the complete letter and all 123 fields. Run that?

In [ ]:
# Send ALL fields this time
prompt_full = f"""You are filling out a graduate school recommendation form.

RECOMMENDER INFO:
{recc_info}

RECOMMENDATION LETTER:
{letter_text}

FORM FIELDS TO FILL:
{json.dumps(fields_to_fill, indent=2)}

For each field, provide the field ID and value to fill. For dropdowns, pick from the options listed.
Return as JSON array: [{{"id": "form_xxx", "value": "..."}}]
Skip fields that are already filled or radio buttons.
"""

print(f"Full prompt length: {len(prompt_full)} chars")
response_full = chat(prompt_full)
print(response_full)

Full prompt length: 40564 chars


Message(id='msg_018Pk65LM1ExVd8AGgXAxUGh', content=[TextBlock(citations=None, text='```json\n[\n  {"id": "form_b2d46a9c-36e6-4095-bf56-9d15b524bea5", "value": "Standard"},\n  {"id": "form_d4a7d23d-9fd3-5542-466e-d2e30fe30e6c", "value": "Belmont University"},\n  {"id": "form_210773b2-503d-7748-63b5-f0f37c83b6ed", "value": "1900 Belmont Blvd"},\n  {"id": "form_cb198687-7870-debf-65f3-8daf444c1a7f", "value": "Nashville"},\n  {"id": "form_1f9293f9-1537-56e3-7bb2-0f4d3a1848a5", "value": "TN"},\n  {"id": "form_a06b225f-d0e7-701d-34c7-e92c5b4d3f3e", "value": "37211"},\n  {"id": "form_c3e60866-2c14-4261-b91a-131821db5dd8", "value": "United States"},\n  {"id": "form_e75dd53c-9b7a-718f-e819-4073d1733059", "value": "scott.hawley@belmont.edu"},\n  {"id": "form_1e4845c3-0bfc-b4ca-dca7-c03fa0835d90", "value": "Outstanding"},\n  {"id": "form_7d37bc49-5a6e-e04c-7417-4d975bdd1248", "value": "Outstanding"},\n  {"id": "form_76fee58a-16a1-9afe-87cc-70b0da52828b", "value": "Outstanding"},\n  {"id": "form_e

Okay, how are we doing? Are we ready to try a sample filling into the form? By the way, it's crucial that you do **not** press the final submit button at the bottom of the form. I need to review the filled-in page manually before we submit!  Note that some of the fields are already filled in. For example, it looks like the first four text fields are already filled in with values. I would say if there are already values filled in, we can skip them and move on to the others. 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Great! Let's parse the response and fill the form. I'll skip pre-filled fields and definitely **not** click submit:

```python
import re

# Extract JSON from response
json_match = re.search(r'```json\s*(.*?)\s*```', response_full.content[0].text, re.DOTALL)
fill_data = json.loads(json_match.group(1))

print(f"Got {len(fill_data)} field mappings to apply")

# Fill the form, skipping pre-filled fields
for item in fill_data:
    field_id = item['id']
    value = item['value']
    
    try:
        elem = page.locator(f'#{field_id}')
        
        # Check if it's a select or input/textarea
        tag = await elem.evaluate('el => el.tagName.toLowerCase()')
        
        # Check if already has a value (skip if so)
        current = await elem.input_value() if tag != 'select' else await elem.evaluate('el => el.value')
        if current and tag != 'select':
            print(f"SKIP (prefilled): {field_id[:30]}")
            continue
        
        if tag == 'select':
            await elem.select_option(label=value)
            print(f"SELECT: {value[:30]}")
        else:
            await elem.fill(value)
            print(f"FILL: {value[:30]}")
            
    except Exception as e:
        print(f"ERROR {field_id[:20]}: {e}")

print("\n✓ Done! Review the form in your browser before submitting.")
```

This will fill the fields and print what it's doing. You can watch it happen in your Chrome window. Ready to try it?

In [ ]:
import re

# Extract JSON from response
json_match = re.search(r'```json\s*(.*?)\s*```', response_full.content[0].text, re.DOTALL)
fill_data = json.loads(json_match.group(1))

print(f"Got {len(fill_data)} field mappings to apply")

# Fill the form, skipping pre-filled fields
for item in fill_data:
    field_id = item['id']
    value = item['value']
    
    try:
        elem = page.locator(f'#{field_id}')
        
        # Check if it's a select or input/textarea
        tag = await elem.evaluate('el => el.tagName.toLowerCase()')
        
        # Check if already has a value (skip if so)
        current = await elem.input_value() if tag != 'select' else await elem.evaluate('el => el.value')
        if current and tag != 'select':
            print(f"SKIP (prefilled): {field_id[:30]}")
            continue
        
        if tag == 'select':
            await elem.select_option(label=value)
            print(f"SELECT: {value[:30]}")
        else:
            await elem.fill(value)
            print(f"FILL: {value[:30]}")
            
    except Exception as e:
        print(f"ERROR {field_id[:20]}: {e}")

print("\n✓ Done! Review the form in your browser before submitting.")

Got 47 field mappings to apply


ERROR form_b2d46a9c-36e6-4: Locator.select_option: Timeout 30000ms exceeded.
Call log:
  - waiting for locator("#form_b2d46a9c-36e6-4095-bf56-9d15b524bea5")
    - locator resolved to <select size="1" autocomplete="off" data-handler-change="1" id="form_b2d46a9c-36e6-4095-bf56-9d15b524bea5" name="form_b2d46a9c-36e6-4095-bf56-9d15b524bea5">…</select>
  - attempting select option action
    2 × waiting for element to be visible and enabled
      - element is not visible
    - retrying select option action
    - waiting 20ms
    2 × waiting for element to be visible and enabled
      - element is not visible
    - retrying select option action
      - waiting 100ms
    45 × waiting for element to be visible and enabled
       - element is not visible
     - retrying select option action
       - waiting 500ms



SKIP (prefilled): form_d4a7d23d-9fd3-5542-466e-d


FILL: 1900 Belmont Blvd


SKIP (prefilled): form_cb198687-7870-debf-65f3-8


...

ERROR form_e86ed888-f537-4: Locator.select_option: Timeout 30000ms exceeded.
Call log:
  - waiting for locator("#form_e86ed888-f537-4188-aa3d-648424e1b3c8")
    - locator resolved to <select size="1" disabled autocomplete="off" required="required" id="form_e86ed888-f537-4188-aa3d-648424e1b3c8" name="form_e86ed888-f537-4188-aa3d-648424e1b3c8">…</select>
  - attempting select option action
    2 × waiting for element to be visible and enabled
      - element is not visible
    - retrying select option action
    - waiting 20ms
    2 × waiting for element to be visible and enabled
      - element is not visible
    - retrying select option action
      - waiting 100ms
    46 × waiting for element to be visible and enabled
       - element is not visible
     - retrying select option action
       - waiting 500ms



ERROR form_6b8f125c-a4eb-4: Locator.select_option: Timeout 30000ms exceeded.
Call log:
  - waiting for locator("#form_6b8f125c-a4eb-4788-a0ca-eb1cc30c724d")
    - locator resolved to <select size="1" disabled autocomplete="off" required="required" id="form_6b8f125c-a4eb-4788-a0ca-eb1cc30c724d" name="form_6b8f125c-a4eb-4788-a0ca-eb1cc30c724d">…</select>
  - attempting select option action
    2 × waiting for element to be visible and enabled
      - element is not visible
    - retrying select option action
    - waiting 20ms
    2 × waiting for element to be visible and enabled
      - element is not visible
    - retrying select option action
      - waiting 100ms
    45 × waiting for element to be visible and enabled
       - element is not visible
     - retrying select option action
       - waiting 500ms



ERROR form_fe61f88a-d93c-4: Locator.select_option: Timeout 30000ms exceeded.
Call log:
  - waiting for locator("#form_fe61f88a-d93c-4df2-a85d-5b43b5d9504a")
    - locator resolved to <select size="1" disabled autocomplete="off" required="required" id="form_fe61f88a-d93c-4df2-a85d-5b43b5d9504a" name="form_fe61f88a-d93c-4df2-a85d-5b43b5d9504a">…</select>
  - attempting select option action
    2 × waiting for element to be visible and enabled
      - element is not visible
    - retrying select option action
    - waiting 20ms
    2 × waiting for element to be visible and enabled
      - element is not visible
    - retrying select option action
      - waiting 100ms
    44 × waiting for element to be visible and enabled
       - element is not visible
     - retrying select option action
       - waiting 500ms



ERROR form_ca4543d9-0318-4: Locator.select_option: Timeout 30000ms exceeded.
Call log:
  - waiting for locator("#form_ca4543d9-0318-4be8-ab15-4b661c9ac54f")
    - locator resolved to <select size="1" disabled autocomplete="off" required="required" id="form_ca4543d9-0318-4be8-ab15-4b661c9ac54f" name="form_ca4543d9-0318-4be8-ab15-4b661c9ac54f">…</select>
  - attempting select option action
    2 × waiting for element to be visible and enabled
      - element is not visible
    - retrying select option action
    - waiting 20ms
    2 × waiting for element to be visible and enabled
      - element is not visible
    - retrying select option action
      - waiting 100ms
    46 × waiting for element to be visible and enabled
       - element is not visible
     - retrying select option action
       - waiting 500ms



CancelledError: 

That worked great. the only thing left is to do the file upload thing I guess for that you would need to know where on my local file system that file is. Now, currently we're running on Solveit, but ultimately my intent is to produce this as a script that can be run from my local machine. in which case that file will be available. 
How about if we focus on packaging up what we've got so far into a nice set of callable functions and then we can fill in the rest of the main function further below.  So the codes that we have above -- the beautifulSoup, the filling in of the form, etc --  let's modularize them as callable functions, yea? 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Exactly! Let's modularize. Here's a clean structure:

```python
from bs4 import BeautifulSoup
import json, re
from claudette import Chat

def scrape_form_fields(html):
    """Extract all fillable form fields from HTML"""
    soup = BeautifulSoup(html, 'html.parser')
    fields = []
    for inp in soup.find_all(['input', 'select', 'textarea']):
        field_id = inp.get('id', '')
        if not field_id: continue
        field_type = inp.get('type', inp.name)
        if field_type in ['hidden', 'submit', 'button']: continue
        
        label = soup.find('label', {'for': field_id})
        label_text = label.get_text(strip=True) if label else ''
        current_value = inp.get('value', '')
        
        options = None
        if inp.name == 'select':
            options = [opt.get_text(strip=True) for opt in inp.find_all('option') if opt.get_text(strip=True)]
        
        fields.append({
            'id': field_id, 'label': label_text, 'type': field_type,
            'options': options, 'prefilled': bool(current_value and field_type not in ['radio','checkbox'] and inp.name != 'select')
        })
    return fields

def get_field_mappings(fields, recc_info, letter_text, model="claude-sonnet-4-20250514"):
    """Use LLM to map recommender info and letter to form fields"""
    prompt = f"""You are filling out a graduate school recommendation form.

RECOMMENDER INFO:
{recc_info}

RECOMMENDATION LETTER:
{letter_text}

FORM FIELDS TO FILL:
{json.dumps([f for f in fields if not f['prefilled']], indent=2)}

For each field, provide the field ID and value to fill. For dropdowns, pick from the options listed.
Return as JSON array: [{{"id": "form_xxx", "value": "..."}}]
Skip radio buttons.
"""
    chat = Chat(model=model)
    response = chat(prompt)
    json_match = re.search(r'```json\s*(.*?)\s*```', response.content[0].text, re.DOTALL)
    return json.loads(json_match.group(1))

async def fill_form(page, mappings, skip_prefilled=True):
    """Fill form fields using Playwright"""
    results = {'filled': [], 'skipped': [], 'errors': []}
    for item in mappings:
        field_id, value = item['id'], item['value']
        try:
            elem = page.locator(f'#{field_id}')
            tag = await elem.evaluate('el => el.tagName.toLowerCase()')
            
            if skip_prefilled and tag != 'select':
                current = await elem.input_value()
                if current:
                    results['skipped'].append(field_id)
                    continue
            
            if tag == 'select':
                await elem.select_option(label=value)
            else:
                await elem.fill(value)
            results['filled'].append(field_id)
        except Exception as e:
            results['errors'].append({'id': field_id, 'error': str(e)[:50]})
    return results

async def upload_recommendation(page, file_path):
    """Upload the recommendation PDF"""
    file_input = page.locator('input[type="file"]').first
    await file_input.set_input_files(file_path)
```

Run that to define the functions, then we can write the main workflow that uses them. Sound good?

In [ ]:
#| export
from bs4 import BeautifulSoup
import json, re


def scrape_form_fields(html):
    """Extract all fillable form fields from HTML"""
    soup = BeautifulSoup(html, 'html.parser')
    fields = []
    for inp in soup.find_all(['input', 'select', 'textarea']):
        field_id = inp.get('id', '')
        if not field_id: continue
        field_type = inp.get('type', inp.name)
        if field_type in ['hidden', 'submit', 'button']: continue
        
        label = soup.find('label', {'for': field_id})
        label_text = label.get_text(strip=True) if label else ''
        current_value = inp.get('value', '')
        
        options = None
        if inp.name == 'select':
            options = [opt.get_text(strip=True) for opt in inp.find_all('option') if opt.get_text(strip=True)]
        
        fields.append({
            'id': field_id, 'label': label_text, 'type': field_type,
            'options': options, 'prefilled': bool(current_value and field_type not in ['radio','checkbox'] and inp.name != 'select')
        })
    return fields

In [ ]:
#| export
from claudette import Chat

def get_field_mappings(fields, recc_info, letter_text, model="claude-sonnet-4-20250514"):
    """Use LLM to map recommender info and letter to form fields"""
    prompt = f"""You are filling out a graduate school recommendation form.

RECOMMENDER INFO:
{recc_info}

RECOMMENDATION LETTER:
{letter_text}

FORM FIELDS TO FILL:
{json.dumps([f for f in fields if not f['prefilled']], indent=2)}

For each field, provide the field ID and value to fill. For dropdowns, pick from the options listed.
Return as JSON array: [{{"id": "form_xxx", "value": "..."}}]
Skip radio buttons.
"""
    chat = Chat(model=model)
    response = chat(prompt)
    json_match = re.search(r'```json\s*(.*?)\s*```', response.content[0].text, re.DOTALL)
    return json.loads(json_match.group(1))

In [ ]:
#| export
async def fill_form(page, mappings, skip_prefilled=True):
    """Fill form fields using Playwright"""
    results = {'filled': [], 'skipped': [], 'errors': []}
    for item in mappings:
        field_id, value = item['id'], item['value']
        try:
            elem = page.locator(f'#{field_id}')
            tag = await elem.evaluate('el => el.tagName.toLowerCase()')
            
            if skip_prefilled and tag != 'select':
                current = await elem.input_value()
                if current:
                    results['skipped'].append(field_id)
                    continue
            
            if tag == 'select':
                await elem.select_option(label=value)
            else:
                await elem.fill(value)
            results['filled'].append(field_id)
        except Exception as e:
            results['errors'].append({'id': field_id, 'error': str(e)[:50]})
    return results

In [ ]:
#| export
async def upload_recommendation(page, file_path):
    """Upload the recommendation PDF"""
    file_input = page.locator('input[type="file"]').first
    await file_input.set_input_files(file_path)

# `formalyzer` CLI script

In [ ]:
#| export
import os 
from fastcore.script import call_parse

@call_parse
def main(pdf_path: str, urls: str, debug: bool = False):
    pdf_path = os.path.expanduser(pdf_path) 
    assert os.path.exists(pdf_path), f"File not found: {pdf_path}"
    if os.path.exists(os.path.expanduser(urls)): 
        if debug: print(f"File {urls} exists. Reading.")
        urls = read_urls_file(urls)
    else: 
        print(f"No file {urls}. Treating it as a single url") 
        urls = [urls] 
    if debug: print("urls =\n",urls)
    letter_text = read_pdf_text(pdf_path)
    if debug: print("letter_text =\n",letter_text) 

In [ ]:
#main("~/recc_letter.pdf","~/recc_urls.txt", debug=True)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

Okay, so above you see my minimal `main` function, but we should add to it now, right? 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Yes! We need to add the Playwright connection and the form-filling loop. Here's what to add:

```python
import os 
from fastcore.script import call_parse
import asyncio

async def process_url(page, url, recc_info, letter_text, pdf_path, debug=False):
    """Process a single recommendation URL"""
    await page.goto(url)
    html = await page.content()
    
    fields = scrape_form_fields(html)
    if debug: print(f"Found {len(fields)} fields")
    
    mappings = get_field_mappings(fields, recc_info, letter_text)
    if debug: print(f"Got {len(mappings)} mappings from LLM")
    
    results = await fill_form(page, mappings)
    if debug: print(f"Filled: {len(results['filled'])}, Errors: {len(results['errors'])}")
    
    await upload_recommendation(page, pdf_path)
    if debug: print("Uploaded PDF")
    
    input("Review the form, then press Enter to continue to next URL (or Ctrl+C to stop)...")
```

The key addition is `process_url()` which handles one form, and you'll need an `async def run_all()` that connects to the browser and loops through URLs.

In [ ]:
#| export
import os 
from fastcore.script import call_parse
import asyncio

async def process_url(page, url, recc_info, letter_text, pdf_path, debug=False):
    """Process a single recommendation URL"""
    await page.goto(url)
    html = await page.content()
    
    fields = scrape_form_fields(html)
    if debug: print(f"Found {len(fields)} fields")
    
    mappings = get_field_mappings(fields, recc_info, letter_text)
    if debug: print(f"Got {len(mappings)} mappings from LLM")
    
    results = await fill_form(page, mappings)
    if debug: print(f"Filled: {len(results['filled'])}, Errors: {len(results['errors'])}")
    
    await upload_recommendation(page, pdf_path)
    if debug: print("Uploaded PDF")
    
    input("Review the form, then press Enter to continue to next URL (or Ctrl+C to stop)...")